In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import requests

In [2]:
def sample_books(book_urls, Author_name,book_name):
    # Create a list to store the DataFrames
    dataframes = []
    # Iterate over the book URLs and labels
    for url, label, title in zip(book_urls, Author_name,book_name):
        # Download book from Gutenberg
        book = requests.get(url)
        book = book.text
        # Split book into a list of words
        words = book.split()
        # Create a list to store the partitions
        partitions = []
        # Use a for loop to sample 200 partitions of 100 words each
        for i in range(200):
            start_index = random.randint(0, len(words) - 100)
            partition = words[start_index : start_index + 100]
            partitions.append(" ".join(partition))
        # Create a Pandas DataFrame to store the partitions
        data = {'text': partitions,'author': [label]*200,'book':title}
        df = pd.DataFrame(data)
        # Append the DataFrame to the list
        dataframes.append(df)
    # Concatenate the DataFrames into a single DataFrame
    final_df = pd.concat(dataframes)
    #return dataframe
    return final_df

In [3]:
book_urls = ['http://www.gutenberg.org/files/1342/1342-0.txt', 'http://www.gutenberg.org/files/84/84-0.txt','https://www.gutenberg.org/files/2701/2701-0.txt','https://www.gutenberg.org/files/1228/1228-0.txt','https://www.gutenberg.org/files/2600/2600-0.txt']
Author_name = ['Jane Austen', 'Mary Wollstonecraft Shelley','Herman Melville', 'Charles Darwin','Leo Tolstoy']
book_name = ['Pride and prejudice', 'Frankenstein','Moby-Dick','The Origin of Species','War and Peace']
df = sample_books(book_urls, Author_name,book_name)

In [4]:
df.to_csv('text_data.csv',index=False)

In [5]:
# Preprocess the data
df['text'] = df['text'].str.lower() # Convert to lowercase
df['text'] = df['text'].str.replace('[^a-zA-Z0-9\s]','') # Remove punctuation

<ipython-input-5-a490076a9bbe>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace('[^a-zA-Z0-9\s]','') # Remove punctuation


In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['author']= le.fit_transform(df['author']) 

In [8]:
df

,text,author,book
0,as puzzle me exceedingly i can readily believe...,2,Pride and prejudice
1,and i never spoke to you without rather wishin...,2,Pride and prejudice
2,i shall be uniformly silent and you may assure...,2,Pride and prejudice
3,inclination any meeting with him would have be...,2,Pride and prejudice
4,replied sir william from that knowledge of wha...,2,Pride and prejudice
...,...,...,...
195,the quiet joyful and amused smile now habitual...,3,War and Peace
196,think that i i understand quite understand i c...,3,War and Peace
197,stood something seemed to be laughing a little...,3,War and Peace
198,looked at him and without replying went on spe...,3,War and Peace


In [9]:
# Split the data into training, validation, and testing sets
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [10]:
# Transform the data into bag-of-words representation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer()
train_matrix = vectorizer.fit_transform(train_df['text'])
test_matrix = vectorizer.transform(test_df['text'])


In [11]:
from sklearn.cluster import KMeans
kmeans = KMeans()
kmeans.fit(train_matrix,y=train_df['author'])

KMeans()

In [12]:
y_pred = kmeans.predict(test_df['author'])

ValueError: Expected 2D array, got 1D array instead:
array=[1. 0. 0. 0. 1. 0. 0. 1. 3. 2. 3. 2. 0. 3. 3. 3. 4. 3. 0. 4. 1. 2. 4. 1.
 4. 4. 2. 3. 3. 3. 4. 2. 0. 1. 4. 2. 0. 1. 3. 2. 4. 2. 4. 3. 4. 4. 3. 0.
 1. 3. 1. 3. 4. 1. 2. 2. 0. 2. 4. 3. 4. 2. 1. 4. 0. 0. 0. 3. 2. 1. 2. 1.
 1. 0. 4. 4. 0. 2. 1. 4. 0. 2. 4. 2. 4. 1. 3. 0. 0. 4. 4. 1. 1. 4. 0. 1.
 0. 2. 2. 3. 0. 4. 3. 4. 0. 0. 1. 3. 2. 3. 1. 3. 3. 3. 2. 1. 2. 3. 1. 4.
 3. 1. 0. 4. 4. 1. 1. 1. 3. 3. 1. 3. 4. 3. 1. 2. 2. 4. 2. 2. 2. 4. 0. 4.
 3. 3. 3. 4. 3. 4. 0. 2. 3. 1. 3. 1. 4. 0. 2. 3. 1. 0. 0. 0. 3. 0. 4. 0.
 4. 4. 4. 1. 2. 3. 1. 4. 4. 4. 3. 0. 1. 0. 4. 1. 3. 2. 4. 4. 3. 0. 3. 3.
 0. 2. 4. 1. 4. 4. 0. 2.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.